In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 625.4/625.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 65.2 MB/s eta 0:00:00
2.19.0-dev20241108


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-11-11 06:19:41--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2024-11-11 06:19:41 (17.0 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2024-11-11 06:19:41--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [ ]:
# Load and preprocess the data
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'text'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])

# Encode labels: ham -> 0, spam -> 1
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

# Adjust parameters for tokenization and padding
vocab_size = 15000  # Increase vocabulary size
max_length = 120    # Increase max length for padding

# Reinitialize the tokenizer and preprocess data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['text'])

train_sequences = tokenizer.texts_to_sequences(train_data['text'])
test_sequences = tokenizer.texts_to_sequences(test_data['text'])

train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Prepare labels
train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Retrain the model
history = model.fit(train_padded, train_labels, epochs=20, validation_data=(test_padded, test_labels), verbose=2)


Epoch 1/20
131/131 - 3s - 25ms/step - accuracy: 0.8655 - loss: 0.3946 - val_accuracy: 0.8657 - val_loss: 0.3713
Epoch 2/20
131/131 - 3s - 20ms/step - accuracy: 0.8660 - loss: 0.3709 - val_accuracy: 0.8657 - val_loss: 0.3644
Epoch 3/20
131/131 - 2s - 13ms/step - accuracy: 0.8660 - loss: 0.3625 - val_accuracy: 0.8657 - val_loss: 0.3551
Epoch 4/20
131/131 - 1s - 9ms/step - accuracy: 0.8660 - loss: 0.3518 - val_accuracy: 0.8657 - val_loss: 0.3412
Epoch 5/20
131/131 - 1s - 10ms/step - accuracy: 0.8660 - loss: 0.3279 - val_accuracy: 0.8657 - val_loss: 0.3146
Epoch 6/20
131/131 - 1s - 9ms/step - accuracy: 0.8689 - loss: 0.2777 - val_accuracy: 0.8807 - val_loss: 0.2276
Epoch 7/20
131/131 - 2s - 12ms/step - accuracy: 0.9189 - loss: 0.1943 - val_accuracy: 0.9468 - val_loss: 0.1592
Epoch 8/20
131/131 - 1s - 9ms/step - accuracy: 0.9598 - loss: 0.1346 - val_accuracy: 0.9763 - val_loss: 0.1210
Epoch 9/20
131/131 - 1s - 10ms/step - accuracy: 0.9756 - loss: 0.0938 - val_accuracy: 0.9763 - val_loss: 0.

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  sequence = tokenizer.texts_to_sequences([pred_text])
  padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')

  prediction_score = model.predict(padded_sequence)[0][0]
  label = 'ham' if prediction_score < 0.5 else 'spam'  # Adjust threshold if necessary

  return [float(prediction_score), label]

# Test the function
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
[0.0028933275025337934, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
You passed the challenge. Great job!
